<h3>Import libraries:</h3>

In [1]:
import roboticstoolbox as rtb
from spatialmath import SE3, base
import matplotlib.pyplot as plt
from math import pi, degrees, radians
import numpy as np
from spatialmath.base import tr2rpy
from mpl_toolkits.mplot3d import Axes3D
import spatialgeometry as sg
import time
import math

# https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file
from ipynb.fs.full.inverse_kinematics_dnb_robot import *

%matplotlib qt

<h3>Define the robot:</h3>

In [2]:
# Define the lenghts
l1 = 10.3
l2 = 16.525
l3 = 15.645
l4 = 8.8
l5 = 7.4

# Adjust the lenghts to plot the robot properly with the library (otherwise, the axes are too small)
# https://la.mathworks.com/matlabcentral/answers/485353-frames-not-appearing-using-robotics-system-toolbox
scaleFactor = 50

l1 /= scaleFactor
l2 /= scaleFactor
l3 /= scaleFactor
l4 /= scaleFactor
l5 /= scaleFactor

# Notes:
# 1. For a revolute joint the theta parameter of the link is ignored, and q used instead.
# 2. For a prismatic joint the d parameter of the link is ignored, and q used instead.
# 3. The joint offset parameter is added to q before computation of the transformation matrix.

# Robot's links
link1 = rtb.RevoluteDH(d=l1, a=0.0, alpha=pi/2, offset=-pi/2, qlim=[0,pi])
link2 = rtb.RevoluteDH(d=0.0, a=l2, alpha=0.0, offset=0.0, qlim=[0,pi])
link3 = rtb.RevoluteDH(d=0.0, a=l3, alpha=0.0, offset=-radians(152), qlim=[0,pi])
link4 = rtb.RevoluteDH(d=0.0, a=l4, alpha=0.0, offset=0.0, qlim=[0,pi])

# https://petercorke.github.io/robotics-toolbox-python/arm_dh.html#roboticstoolbox.robot.DHLink.PrismaticDH

# Create the robot
robot = rtb.DHRobot(
    links=[link1, link2, link3, link4],
    name="D&B Robot",
    tool = SE3.Ty(-l5)
    )

# Home angles
qHome = [radians(90), radians(90), radians(0), radians(62)]

# Set home position
robot.addconfiguration_attr('home',qHome)

print(robot)
print(f'Robot is in 1:{scaleFactor} scale.')

DHRobot: D&B Robot, 4 joints (RRRR), dynamics, standard DH parameters
┌────────────┬───────┬────────┬───────┬──────┬────────┐
│     θⱼ     │  dⱼ   │   aⱼ   │  ⍺ⱼ   │  q⁻  │   q⁺   │
├────────────┼───────┼────────┼───────┼──────┼────────┤
│  q1 - 90°  │ 0.206 │      0 │ 90.0° │ 0.0° │ 180.0° │
│  q2        │     0 │ 0.3305 │  0.0° │ 0.0° │ 180.0° │
│  q3 - 152° │     0 │ 0.3129 │  0.0° │ 0.0° │ 180.0° │
│  q4        │     0 │  0.176 │  0.0° │ 0.0° │ 180.0° │
└────────────┴───────┴────────┴───────┴──────┴────────┘

┌──────┬───────────────────────────────────────┐
│ tool │ t = 0, -0.15, 0; rpy/xyz = 0°, 0°, 0° │
└──────┴───────────────────────────────────────┘

┌──────┬──────┬──────┬─────┬──────┐
│ name │ q0   │ q1   │ q2  │ q3   │
├──────┼──────┼──────┼─────┼──────┤
│ home │  90° │  90° │  0° │  62° │
└──────┴──────┴──────┴─────┴──────┘

Robot is in 1:50 scale.


In [3]:
# Plot home position
fig1 = plt.figure(figsize=(8, 8))  # Adjust this for larger or smaller axes
robot.plot(q=robot.home, backend='pyplot', fig = fig1)
plt.show()

<h3>Kinematics tests:</h3>

In [4]:
# Compute the forward kinematics
fk1 = robot.fkine(qHome)

# Get the real coordinates
fk1Escaled = SE3.Rt(fk1.R, fk1.t * scaleFactor)
# This SE3.Rt method takes a rotation matrix R and a translation vector t and creates the desired SE(3) transformation

print(fk1Escaled)

   1         0         0         16.14     
   0         0        -1         0         
   0         1         0         5.611     
   0         0         0         1         



In [5]:
# Compute the inverse kinematics
ik1 = robot.ikine_LM(fk1, joint_limits = True)

for i in range(len(ik1.q)):
    print(f'q{i+1} = {round(degrees(ik1.q[i]),2)}°.')

q1 = 90.0°.
q2 = 90.0°.
q3 = 0.0°.
q4 = 62.0°.


In [6]:
# We'll use our own inverse kinematics function which gives us easier control of the end effector's orientation

pxHome = fk1.A[:,3][0]
pyHome = fk1.A[:,3][1]
pzHome = fk1.A[:,3][2]

# The orientation of the end effector is defined with respect to the base frame (only the pitch angle can be controlled)
pitchAngle = radians(0)

qList = inverse_kinematics(pxHome, pyHome, pzHome, pitchAngle, scaleFactor)

for index,q in enumerate(qList):
    print(f'q{index+1}: {q}°.')

q1: 90.0°.
q2: 90.0°.
q3: 0.0°.
q4: 62.0°.


In [7]:
# Traslation vector and pitch angle for frame1
pxFrame1 = 19/scaleFactor
pyFrame1 = -8.2/scaleFactor
pzFrame1 = 0.2/scaleFactor
pitchAngleFrame1 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qFrame1 = inverse_kinematics(pxFrame1, pyFrame1, pzFrame1, pitchAngleFrame1, scaleFactor)
TFrame1 = robot.fkine([radians(i) for i in qFrame1])

print('Joint angles required to achieve the pose (orientation and translation) of frame1:')
print(qFrame1)

print('\nHomogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:')
print(TFrame1)

# Extract RPY angles (mobile/current frame, ZYX convention)
rpyFrame1 = tr2rpy(TFrame1.R, order='zyx', unit='deg')

print('Euler angles in the ZYX convention with respect to the mobile/current frame:')
print(f'- Roll: {round(rpyFrame1[0],2)}°.')
print(f'- Pitch: {round(rpyFrame1[1],2)}°.')
print(f'- Yaw: {round(rpyFrame1[2],2)}°.')

# mobile/current frame, ZYX convention = fixed frame, XYZ convention
# https://mecademic.com/insights/academic-tutorials/space-orientation-euler-angles/
# https://bdaiinstitute.github.io/spatialmath-python/func_3d.html#spatialmath.base.transforms3d.tr2rpy

Joint angles required to achieve the pose (orientation and translation) of frame1:
[66.66, 51.15, 16.45, 84.39]

Homogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:
   0.9182    0.0001603 -0.3962    0.38      
  -0.3962   -6.915e-05 -0.9182   -0.164     
  -0.0001745  1         0         0.003953  
   0         0         0         1         

Euler angles in the ZYX convention with respect to the mobile/current frame:
- Roll: 90.0°.
- Pitch: 0.01°.
- Yaw: -23.34°.


In [8]:
# Calculate the geometric jacobian with respect to the base frame
jacob1 = robot.jacob0(qFrame1)
print(jacob1)

[[ 6.19890728e-02  2.10426531e-01  4.83584738e-02 -1.21303790e-01]
 [ 5.08037414e-02 -2.56755609e-01 -5.90054369e-02  1.48010939e-01]
 [-7.65223838e-19 -8.01477716e-02 -2.89573330e-01 -1.27507460e-01]
 [-6.93889390e-17  7.73434764e-01  7.73434764e-01  7.73434764e-01]
 [ 0.00000000e+00  6.33875907e-01  6.33875907e-01  6.33875907e-01]
 [ 1.00000000e+00  6.12323400e-17  6.12323400e-17  6.12323400e-17]]


In [9]:
# Calculate the geometric jacobian with respect to the end-effector frame
jacob2 = robot.jacobe(qFrame1)
print(jacob2)

[[ 2.60208521e-18  4.27297446e-02  2.79292000e-01  1.48000000e-01]
 [-2.77555756e-17 -3.38822375e-01 -1.08022571e-01  1.76000000e-01]
 [ 8.01477716e-02 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.93796042e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.11217922e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.12323400e-17  1.00000000e+00  1.00000000e+00  1.00000000e+00]]


In [10]:
# Calculate the analytical jacobian with respect to the base frame
jacob3 = robot.jacob0_analytical(qFrame1, representation='rpy/zyx')
print(jacob3)

[[ 6.19890728e-02  2.10426531e-01  4.83584738e-02 -1.21303790e-01]
 [ 5.08037414e-02 -2.56755609e-01 -5.90054369e-02  1.48010939e-01]
 [-7.65223838e-19 -8.01477716e-02 -2.89573330e-01 -1.27507460e-01]
 [ 3.95475620e-16  6.21724894e-15  6.21724894e-15  6.21724894e-15]
 [ 5.36678177e-17 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  6.21724894e-15  6.21724894e-15  6.21724894e-15]]


<h3>Trajectory test:</h3>

In [11]:
# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.trapezoidal

# https://la.mathworks.com/help/robotics/ref/trapveltraj.html
# https://la.mathworks.com/help/robotics/ug/choose-trajectories-for-manipulator-paths.html
# https://la.mathworks.com/help/robotics/ug/design-a-trajectory-with-velocity-limits-using-a-trapezoidal-velocity-profile.html
# https://medium.com/mathworks/trajectory-planning-for-robot-manipulators-522404efb6f0

# https://youtu.be/f3JVWqsDhbE?si=McvOoeyu82TyQrwv
# https://youtu.be/Fd7wjZDoh7g?si=ZXw3PyyvvAwSEzFu

# Servomotor MG996R ----> Operating speed: 0.17 s/60º (4.8 V), 0.14 s/60º (6 V) 
# 0.14 s/60º (6 V) --------> 7,14 °/s (max speed)

# Set the time required to complete the trajectory
timeStepsTraj1 = 100
totalTime = 10 # Seconds
timeTraj1 = np.linspace(0, totalTime, timeStepsTraj1)

In [12]:
# Compute the trajectory using a trapezoidal velocity profile
# tg = rtb.tools.trapezoidal(pxHome*scaleFactor, pxFrame1*scaleFactor, timeTraj1)
# tg.plot()

# Generate the trajectory using a trapezoidal interpolator
q0Traj1 = [i*scaleFactor for i in [pxHome, pyHome, pzHome]]
q1Traj1 = [i*scaleFactor for i in [pxFrame1, pyFrame1, pzFrame1]]
Traj1 = rtb.tools.trajectory.mtraj(rtb.tools.trapezoidal, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1.plot()

In [13]:
# Create a new figure for 3D plotting
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1.s[:,0], Traj1.s[:,1], Traj1.s[:,2])

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a trapezoidal interpolator

In [14]:
# Let's repeat the same process using a quintic polynomial interpolator

# Generate the trajectory using a quintic polynomial interpolator
Traj1v2 = rtb.tools.trajectory.mtraj(rtb.tools.quintic, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1v2.plot()

In [15]:
# Create a new figure for 3D plotting
fig3 = plt.figure()
ax = fig3.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v2.s[:,0], Traj1v2.s[:,1], Traj1v2.s[:,2])

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a quintic polynomial interpolator

In [16]:
# Let's do it again with initial and final velocity different from zero (impractical in this case, it's just for testing)

# Set the velocities (they could be different for each axis)
dq0Traj1v3 = 0.5
dq1Traj1v3 = 1

Traj1v3x = rtb.tools.trajectory.quintic(q0Traj1[0], q1Traj1[0], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3y = rtb.tools.trajectory.quintic(q0Traj1[1], q1Traj1[1], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3z = rtb.tools.trajectory.quintic(q0Traj1[2], q1Traj1[2], timeTraj1, dq0Traj1v3, dq1Traj1v3)

# Create a new figure for 3D plotting
fig4 = plt.figure()
ax = fig4.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v3x.s, Traj1v3y.s, Traj1v3z.s)

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We don't get a line in the cartesian space using a polynomial interpolator if the initial and final velocities are different from zero
# This also happens between viapoints

In [17]:
# We've only interpolated the position of the robot, but it's also required an orientation interpolation
# Position and orientation interpolation are two different issues, therefore they're done separately

# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.ctraj
# We'll handle position and orientation interpolation through the ctraj function

# Set the time required to complete the trajectory
timeStepsTraj2 = 20
totalTime2 = 30 # Seconds
timeTraj2 = np.linspace(0, totalTime2, timeStepsTraj2)

# Compute the Cartesian trajectory between two poses (position + orientation)
Traj2 = rtb.tools.trajectory.ctraj(fk1, TFrame1, timeTraj2)

# Afterwards, compute the inverse kinematics for each pose along the trajectory
# These joint configurations can be fed into a control system to execute and control the movements of the robotic arm

# qTraj2 = robot.ikine_LM(Traj2)

# Plot the trajectory with the library
# robot.plot(qTraj2.q, backend='pyplot')
# plt.show()

<h3>Resolved-Rate Motion Control:</h3>

In [18]:
# https://github.com/jhavl/dkt/blob/main/Part%201/3%20Resolved-Rate%20Motion%20Control.ipynb
# https://arxiv.org/pdf/2207.01796

# https://youtu.be/sc5DZOkNvTc?si=JpxpIKR4r8iLLvUk
# https://youtu.be/1GaxV8x4YHA?si=SVk1Yh55wbxU7_Xb
# https://youtu.be/AgQ_Bybl_9k?si=KLwMJ_HOi7Fy8zhD

<h4>Method 1:</h4>

In [19]:
# https://youtu.be/-TUTqVOGSa8?si=3OldHsu_qE9i22FQ
# https://youtu.be/s0NC85TAPaA?si=zr1ObUN9i66GhVMp
# https://youtu.be/axEadpYSOv8?si=grupjc4v4PHLdA9d

def error(currentPose, desiredPose):

    # Returns the error between the desired and the current pose (translation: vector, rotation: angle-axis notation)
    # currentPose and desiredPose --> np.ndarray with shape = (4,4)
    # currentPose and desiredPose are the poses of the end effector relative to the base frame
    
    # Error array
    e = np.empty(6)

    # The position error
    e[:3] = desiredPose[:3, -1] - currentPose[:3, -1]

    # The rotation error expressed as rotation matrix
    R = desiredPose[:3, :3] @ currentPose[:3, :3].T

    # Transform the rotation matrix to its Euler vector equivalent
    
    li = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

    if np.linalg.norm(li) < 1e-6:
        # If li is a zero vector (or very close to it)

        # Diagonal matrix case
        if np.trace(R) > 0:
            # (1,1,1) case
            a = np.zeros((3,))
        else:
            a = np.pi / 2 * (np.diag(R) + 1)
    else:
        # Non-diagonal matrix case
        ln = np.linalg.norm(li)
        a = math.atan2(ln, np.trace(R) - 1) * li / ln
    
    # Append the rotation error expressed in the angle-axis representation in the error array
    e[3:] = a

    # Return the error
    return e

In [20]:
def p_servo(currentPose, desiredPose, gain, threshold = 0.01):
    
    # Position-based servoing

    # Returns the end-effector velocity in the cartesian space which will cause the robot to approach the desired pose

    # currentPose: The current pose of the end-effecor in the base frame (ndarray)
    # desiredPose: The desired pose of the end-effecor in the base frame (ndarray)
    # gain: The gain for the proportional controller. A vector corresponding to each Cartesian axis (array-like)
    # threshold: The threshold or tolerance of the final error between the robot's pose and desired pose (float)

    # v: The velocity of the end-effector which will casue the robot to approach desiredPose (ndarray(6))
    # arrived: True if the robot is within the threshold of the final pose (bool)
    
    if isinstance(currentPose, SE3):
        currentPose = currentPose.A

    if isinstance(desiredPose, SE3):
        desiredPose = desiredPose.A

    # Calculate the pose error vector
    e = error(currentPose, desiredPose)

    # Construct our gain diagonal matrix
    if base.isscalar(gain):
        k = gain * np.eye(6)
    else:
        k = np.diag(gain)

    # Calculate our desired end-effector velocity
    v = k @ e

    # Check if we have arrived
    arrived = True if np.sum(np.abs(e)) < threshold else False

    # Tuple[np.ndarray, bool]
    return v, arrived

In [27]:
# https://github.com/petercorke/robotics-toolbox-python/blob/master/roboticstoolbox/examples/RRMC.py

env = rtb.backends.PyPlot.PyPlot()
env.launch('Resolved-Rate Motion Control Example')

arrived = False

env.add(robot)

# Set the current pose
robot.q = qHome

# Desired pose
pxFrame2 = 37/scaleFactor
pyFrame2 = -8.2/scaleFactor
pzFrame2 = 0.2/scaleFactor
pitchAngleFrame2 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qFrame2 = inverse_kinematics(pxFrame2, pyFrame2, pzFrame2, pitchAngleFrame2, scaleFactor)
TFrame2 = robot.fkine([radians(i) for i in qFrame2])

# Proportional control gain
gain = 2

# Time step
dt = 0.05

# Lists to save the values obtained in each time step
posesObtained = []
cartesianVelocitiesObtained = []
jointPositionsObtained = []
jointVelocitiesObtained = []

while not arrived:

    start = time.time()
    
    # HTM involved
    currentPose = robot.fkine(robot.q)
    desiredPose = TFrame2
    
    # Cartesian velocity required
    v, arrived = p_servo(currentPose, desiredPose, gain)
    
    # Map it to joint velocities using the geometric jacobian relative to the base frame
    qd = np.linalg.pinv(robot.jacob0(robot.q)) @ v
    
    # Apply the calculated joint velocity to the robot's actuators for movement
    robot.qd = qd
    
    # Append the values obtained in this iteration
    posesObtained.append(currentPose)
    cartesianVelocitiesObtained.append(v)
    jointPositionsObtained.append(robot.q)
    jointVelocitiesObtained.append(qd)
    
    env.step(dt)
    stop = time.time()
    
    if stop - start < dt:
        time.sleep(dt - (stop - start))

# robot.plot(q=[radians(i) for i in qFrame2], backend='pyplot')

In [31]:
# Create a new figure for 3D plotting
fig5 = plt.figure()
ax = fig5.add_subplot(111, projection='3d')

# Plot the 3D trajectory for the translation motion in the workspace
for htm in posesObtained:
    ax.scatter(htm.A[0,3], htm.A[1,3], htm.A[2,3])

# Plot the desired pose
ax.scatter(TFrame2.A[0,3], TFrame2.A[1,3], TFrame2.A[2,3], marker="*", linewidths=3, c='r')

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

<h4>Method 2:</h4>